## Apply the model using Video

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import imutils
from imutils.video import VideoStream

#### define a function to detect the person in the video is wearing a musk or not

In [2]:
def detect_and_predict_mask(frame,faceNet,maskNet):
    #grab the dimensions of the frame and then consturct a blob
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0))
    
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    #initialize our list of faces, their corrseponding locations and list of predictions
    
    faces = []
    locs = []
    preds = []
    
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
    
        if confidence > 0.5 :
            #we need the X,Y coordinates
            box = detections[0,0,i,3:7] * np.array([w,h,w,h])
            (startX,startY,endX,endY) = box.astype('int')
        
            #ensure the bounding boxes fall within the dimensions of the framemax
            (startX,startY) = (max(0,startX), max(0,startY))
            (endX,endY) = (min(w - 1,endX), min(h - 1,endY))
        
            #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face = cv2.resize(face,(224,224))
            face = img_to_array(face)
            face = preprocess_input(face)
        
            faces.append(face)
            locs.append((startX,startY,endX,endY))
        
        #only make a predictions if atleast one face was detected
        if len(faces) > 0:
            faces = np.array(faces,dtype='float32')
            preds = maskNet.predict(faces, batch_size = 12)
    
    return (locs, preds)
        

In [3]:
prototxtPath = os.path.sep.join([r'C:\Users\Nirmalya Majhi\Desktop\Fash Mask Detection\Face Detector','deploy.prototxt'])
weightsPath = os.path.sep.join([r'C:\Users\Nirmalya Majhi\Desktop\Fash Mask Detection\Face Detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [4]:
faceNet = cv2.dnn.readNet(prototxtPath,weightsPath)

In [5]:
maskNet = load_model(r'C:\Users\Nirmalya Majhi\Desktop\Fash Mask Detection\mobilenet_v2_model.h5')

#### Python code for mask detection

In [6]:
vs = VideoStream(src=0).start()

while True:
    #grab the frame from the threaded video stream and resize it
    #to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame,width=400)
    
    #detect faces in the frame and predict if they wearing masks or not
    (locs,preds) = detect_and_predict_mask(frame,faceNet,maskNet)
    
    #loop over the detectedface locations and their corrosponding locations
    for (box,pred) in zip(locs,preds):
        (startX,startY,endX,endY) = box
        (mask,withoutMask) = pred
        
        #determine the class label and color we will use to draw the bounding box and text
        label = 'Mask' if mask > withoutMask else 'No Mask'
        color = (0,255,0) if label == 'Mask' else (0,0,255)
        
        #display the label and bounding boxes
        cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
        
    #show the output frame
    cv2.imshow("Frame",frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
        
cv2.destroyAllWindows()
vs.stop()

1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 68ms/step


1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 40ms/step


...

1/1 [==============================] - 0s 64ms/step


1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 47ms/step
